## Zeitreihenanalyse Covid-19

Datenquelle: Bereinigte Daten der Johns Hopkins Unviversity

Analyse der Entwicklung Neuinfektionen Covid_19 Summe letzte 7 Tage je 100.000 Einwohner

In [1]:
import pandas as pd 
import folium
from selenium import webdriver
import time
import os

Hier holen wir uns die bereinigten Daten der John-Hopskins-University.

In [2]:
covid_df = pd.read_csv('https://raw.githubusercontent.com/Lucas-Czarnecki/COVID-19-CLEANED-JHUCSSE/master/COVID-19_CLEAN/csse_covid_19_time_series_cleaned/time_series_covid19_cases_tidy.csv',parse_dates=['Date'])

covid_df = covid_df.drop(columns=['Province_State', 'Latitude', 'Longitude'])

covid_df.rename(columns=dict(Country_Region = 'land', Confirmed = 'infiziert', Deaths = 'tot', Recovered = 'erholt', Date = 'datum'),inplace=True)

Jetzt holen wir uns die Ländercodes (ISO 3-stellig) aller Länder

In [3]:
country_codes = pd.read_csv('https://raw.githubusercontent.com/datasets/country-codes/master/data/country-codes.csv')
country_codes.rename(columns={'ISO3166-1-Alpha-3':'iso3', 'official_name_en':'iso_land'},inplace=True)
country_codes = country_codes[['iso3','iso_land']]

Leider muss man etliche Ländernamen anpassen, damit die Tabellen synchron sind.

In [4]:
convert = {'United Kingdom of Great Britain and Northern Ireland':'United Kingdom', 'Bolivia (Plurinational State of)':'Bolivia', 'Brunei Darussalam':'Brunei', 'Myanmar':'Burma', 'Congo':'Congo (Brazzaville)', 'Democratic Republic of the Congo':'Congo (Kinshasa)', 'Iran (Islamic Republic of)':'Iran', 'Republic of Korea':'South Korea', "Lao People's Democratic Republic":'Laos', 'Republic of Moldova':'Moldova', 'Russian Federation':'Russia','Syrian Arab Republic':'Syria', 'United Republic of Tanzania':'Tanzania','United States of America':'United States','Venezuela (Bolivarian Republic of)': 'Venezuela', 'Viet Nam':'Vietnam'}
old = convert.values()
new = convert.keys()
covid_df.land = covid_df.land.replace(old,new)


In [5]:
not_in_country_codes = covid_df[~covid_df.land.isin(country_codes.iso_land)]
not_in_country_codes.land.unique()

array(["Cote d'Ivoire", 'Diamond Princess', 'Kosovo', 'MS Zaandam',
       'North Macedonia', 'Taiwan', 'West Bank and Gaza'], dtype=object)

Hier findet das Zusammenführen der ISO-Codes mit der COVID-Tabelle statt

In [6]:
covid_df = pd.merge(covid_df, country_codes, left_on=['land'], right_on=['iso_land'])


In [7]:
pop_df = pd.read_csv('https://raw.githubusercontent.com/datasets/population/master/data/population.csv')

pop_df = pop_df[pop_df.Year == 2018]
pop_df = pop_df.append({'Country Name':'Eritrea', 'Value': 5_100_000, 'Country Code':'ERI'},ignore_index=True)

Über die die ISO-Codes
 erfolgt eine Zusammenführung der Covid-Tabelle mit den Bevölerkungszahlen. Nicht sehr speichereffizient, da jede Zeile des Landes für jeden Tag jetzt auch die Bevölkerungszahl enthält, aber dafür einfach in der weiteren Berechnung von Infizierten/100.000 Einwohner

In [8]:
covid_df = pd.merge(covid_df, pop_df, left_on=['iso3'], right_on=['Country Code'])
covid_df = covid_df.drop(columns=['iso_land', 'Country Name', 'Country Code', 'Year'])

Jetzt ermitteln wir die Neu Infizierten kumuliert über die letzten 7 Tage und umgerechnet auf 100.000 Einwohner

In [9]:
# Berechnung von weiteren Daten auf der Basis der vorhandenen Informationen
covid_df['neuInfiziert'] = covid_df.infiziert.diff(7) / covid_df.Value * 100_000
covid_df.tail()

,land,datum,infiziert,tot,erholt,iso3,Value,neuInfiziert
49339,Zimbabwe,2020-07-27,2704,36,542.0,ZWE,14439018,6.863348
49340,Zimbabwe,2020-07-28,2817,40,604.0,ZWE,14439018,6.904902
49341,Zimbabwe,2020-07-29,2879,41,887.0,ZWE,14439018,5.852199
49342,Zimbabwe,2020-07-30,3092,53,924.0,ZWE,14439018,6.704057
49343,Zimbabwe,2020-07-31,3169,67,1004.0,ZWE,14439018,6.046118


Hier holen wir uns die GEO-JSON-Daten der Umrisse der einzelnen Länder. Dann ermitteln wir alle ISO-Ländercodes von Ländern aus der GEO-JSON-Tabelle und schmeißen danach alle Länder aus der COVID-Datenanalyse, die nicht in der GEO-JSON sind.

Wenn wir keinen Umriss haben, brauchen wir uns um die Daten dieses Landes nicht zu kümmern.

In [10]:
geo_iso = set()
geo_json = pd.read_json('https://raw.githubusercontent.com/austinkeeley/Countries-GeoJSON/master/countries.json')
for row in geo_json.features:
    geo_iso.add(row['properties']['iso_a3'])
covid_df = covid_df[covid_df.iso3.isin(geo_iso)]   

Hier fliegen jetzt alle Länder raus, deren Einwohnerzahl unter 3 Mio. liegen. Dadurch wird verhindert, dass irgenwelche kleinen Länder mit Ihren Infektionszahlen (schönes Beispiel "Lichtenstein") den Schnitt weltweit dominieren.

In [11]:
covid_df = covid_df[covid_df.Value > 3_000_000]   

Jetzt wird für jedes Datum in covid_df eine eigene Map erzeugt und als HTML im Unterverzeichnis Teil_35_Maps unter dem Namen Map<3-stellige-fortlaufende Nr, beginnend mit 7>.html gespeichert

In [12]:
counter = counter_start = 170
geo_data_raw = r'https://raw.githubusercontent.com/austinkeeley/Countries-GeoJSON/master/countries.json'
for date in covid_df.datum.unique()[counter_start:]:
    df = covid_df[covid_df.datum == date]
    d = pd.to_datetime(str(date))
    d = d.strftime('%d.%m.%Y')
    title_html = f'<h3 align="center" style="font-size:20px"><b>Neuinfektioen Covid-19 vom {d}</b></h3>'
    m = folium.Map(location = [12.2395, -1.5584094], zoom_start=3)
    m.choropleth(
        geo_data= geo_data_raw,
        data= df,
        columns=['iso3', 'neuInfiziert'],
        key_on = 'feature.properties.iso_a3',
        fill_color = 'YlOrRd',
        nan_fill_color = 'white',
        fill_opacity = 0.7,
        line_opacity = 0.2,
        legend_name = 'Neuinfektionen summiert über die letzten 7 Tage je 100.000 Einwohner',
        highlight = True
    )
    m.get_root().html.add_child(folium.Element(title_html))
    m.save(f'Teil_35_Maps\World{counter:03d}.html')
    counter += 1

Und hier die Umwandlung von HTML auf PNG. Durch den Aufruf der html-seite in Selenium und export als PNG (sic!)

In [13]:
delay=5
browser = webdriver.Chrome('d:\programme\Selenium_web_driver\chromedriver.exe')
for file in os.listdir('Teil_35_Maps'):
    if not file.endswith('.html') or not file.startswith('World') or int(file[5:8]) < counter_start: continue
    browser.get(f'file:///D:/daten/Sprachen/Python/A-beautiful-code-in-Python/Teil_35_Maps/{file}')
    #Give the map tiles some time to load
    time.sleep(5)
    #Grab the screenshot
    browser.save_screenshot(f'D:/daten/Sprachen/Python/A-beautiful-code-in-Python/Teil_35_Maps/{file}.png')
#Close the browser
browser.quit()